In [1]:
import os
df="C:\\Users\\smartm13\\Desktop\\free codes\\herc-data\\herc-data"
catgs=[df+"\\"+f for f in os.listdir(df)]
files=sum([[(f+"\\"+ff,f.split('\\')[-1]) for ff in os.listdir(f)] for f in catgs],[])
print(len(catgs),"categories and",len(files),"files read.")

32 categories and 8840 files read.


In [2]:
print("locha="+str(len((set([f[0].split('_')[-2].split("\\")[-1] for f in files]))-set([c.split('\\')[-1] for c in catgs]))))

locha=0


In [15]:
from PIL import Image,ImageOps
def forCsvRow(fileadr,new_width=24,new_height=24):
    img = Image.open(fileadr)
    img = ImageOps.invert(img).resize((new_width, new_height), Image.ANTIALIAS)
    return list(img.getdata())
def genCSVs(files):
    csv=[]
    for fadr,fcatg in files:
        csv.append([fcatg]+forCsvRow(fadr))
    return csv
def genCSVs(files):
    for fadr,fcatg in files:
        yield [fcatg]+forCsvRow(fadr)


In [17]:
import csv
with open("emnist_data_op.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(genCSVs(files[:]))
print('check, file writing done.')

In [4]:
import pandas as pd,numpy as np
wdata=pd.read_csv(r"emnist_data_op.csv")
 
catgs=wdata.iloc[:,0].tolist()
catgSet=list(set(catgs))
catgf=[catgSet.index(x) for x in catgs]
wdata.iloc[:,0]=catgf
 
from sklearn.model_selection import train_test_split
 
train_db,test_db=train_test_split(wdata,random_state=42,train_size=.6)

c:\users\smartm13\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model
 
num_classes = len(catgSet)
y_train = train_db.iloc[:,0]
y_train = np_utils.to_categorical(y_train, num_classes)
print ("y_train:", y_train.shape)
 
x_train = train_db.iloc[:,1:]
x_train = x_train.astype('float32')
x_train /= 255
print ("x_train:",x_train.shape)
 
inp = Input(shape=(len(wdata.columns)-1,))
hidden_1 = Dense(1024, activation='relu')(inp)
dropout_1 = Dropout(0.2)(hidden_1)
out = Dense(num_classes, activation='softmax')(hidden_1)
model = Model(input=inp, output=out)
 
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
 
model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=512, nb_epoch=10,
          verbose=2, validation_split=0.1) # ...holding out 10% of the data for validation
 
y_test = test_db.iloc[:,0]
y_test = np_utils.to_categorical(y_test, num_classes)
print ("y_test:", y_test.shape)
 
x_test = test_db.iloc[:,1:]
x_test = x_test.astype('float32')
x_test /= 255
print ("x_test:",x_train.shape)
 
print(model.evaluate(x_test, y_test, verbose=2)) # Evaluate the trained model on the test set!

y_train: (5303, 32)
x_train: (5303, 576)


c:\users\smartm13\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
c:\users\smartm13\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 4772 samples, validate on 531 samples
Epoch 1/10
 - 8s - loss: 3.2708 - acc: 0.2982 - val_loss: 2.9653 - val_acc: 0.4407
Epoch 2/10
 - 0s - loss: 2.7010 - acc: 0.4834 - val_loss: 2.3778 - val_acc: 0.4934
Epoch 3/10
 - 0s - loss: 2.1123 - acc: 0.5704 - val_loss: 1.8688 - val_acc: 0.6121
Epoch 4/10
 - 0s - loss: 1.6481 - acc: 0.6565 - val_loss: 1.4899 - val_acc: 0.6742
Epoch 5/10
 - 0s - loss: 1.3193 - acc: 0.7330 - val_loss: 1.2305 - val_acc: 0.7571
Epoch 6/10
 - 0s - loss: 1.0967 - acc: 0.7816 - val_loss: 1.0520 - val_acc: 0.7797
Epoch 7/10
 - 0s - loss: 0.9351 - acc: 0.8139 - val_loss: 0.9140 - val_acc: 0.8079
Epoch 8/10
 - 0s - loss: 0.8143 - acc: 0.8363 - val_loss: 0.8143 - val_acc: 0.8267
Epoch 9/10
 - 0s - loss: 0.7187 - acc: 0.8510 - val_loss: 0.7415 - val_acc: 0.8399
Epoch 10/10
 - 0s - loss: 0.6444 - acc: 0.8659 - val_loss: 0.6851 - val_acc: 0.8418
y_test: (3536, 32)
x_test: (5303, 576)
[0.7067755360948554, 0.8407805429864253]
